In [51]:
import pygsti
from scipy.linalg import expm
import numpy as np
from pygsti.tools import unitary_to_superop
from pygsti.modelpacks import smq1Q_XYZI
from matplotlib import pyplot as plt
from pygsti.circuits import Circuit
from quapack.pyRPE import RobustPhaseEstimation
from quapack.pyRPE.quantum import Q as _rpeQ
from tqdm import tqdm
from importlib import reload

In [52]:
def PauliMatrix(idx):
    if idx == 0:
        return np.eye(2)
    elif idx == 1:
        return np.array([[0, 1], [1, 0]])
    elif idx == 2:
        return np.array([[0, -1j], [1j, 0]])
    elif idx == 3:
        return np.array([[1, 0], [0, -1]])
    else:
        raise ValueError('Invalid index for Pauli matrix')
    
def PauliTensor(idx1, idx2):
    return np.kron(PauliMatrix(idx1), PauliMatrix(idx2))

def make_generator(pauli):
    dimension = pauli.shape[0]
    I = np.eye(dimension)
    return -(1j/2)*(np.kron(I, pauli) - np.kron(pauli.T, I))

def decimal_to_quaternary(n):
    if n == 0:
        return '0'
    quaternary = ''
    while n:
        n, remainder = divmod(n, 4)
        quaternary = str(remainder) + quaternary
    return quaternary

def decimal_to_pauli_string(idx, num_qubits):
    quaternary = decimal_to_quaternary(idx).zfill(num_qubits)
    return ''.join([['I', 'X', 'Y', 'Z'][int(digit)] for digit in quaternary])

def decimal_to_pauli_tensor(idx, num_qubits):
    quaternary = decimal_to_quaternary(idx).zfill(num_qubits)
    tensor = 1
    for digit in quaternary:
        tensor = np.kron(tensor, PauliMatrix(int(digit)))
    return tensor

def unitary_to_pmat(U):
    return np.kron(U.conj(), U)

In [53]:
zz_generator = make_generator(PauliTensor(3, 3))
zz_pmat = expm((np.pi/2)*zz_generator)

In [54]:
def make_sorted_eigsystem(pmat):
    eigvals, eigvecs = np.linalg.eig(pmat)
    idxs = np.argsort(np.angle(eigvals))
    return eigvals[idxs], eigvecs[:, idxs]

def make_perturbation_vector(pmat, generator):
    eigvals, eigenvectors = make_sorted_eigsystem(pmat)
    perturbations = []
    for e in eigenvectors:
        p = np.conj(e).T @ generator @ e
        perturbations.append(p)
    return np.array(perturbations)

In [55]:
def make_pertubation_dict(pmat, generators, interleaving_operators):
    pertubation_dict = {}
    for k, v in interleaving_operators.items():
        pertubation_dict[k] = {}
        for kg, g in generators.items():
            pertubation_dict[k][kg] = make_perturbation_vector(pmat@v, g)
    return pertubation_dict

In [56]:
import itertools

In [57]:
# make the set of all single qubit clifford rotations
clifford_generators = {
    'ri': np.eye(4),
    'rx': expm((np.pi/2)*make_generator(PauliMatrix(1))), 
    'ry': expm((np.pi/2)*make_generator(PauliMatrix(2))),
    'rz': expm((np.pi/2)*make_generator(PauliMatrix(3)))
}
num_products = 4
all_product_strings = [i for i in itertools.product(clifford_generators.keys(), repeat=num_products)]
complete_dict = {}
for product in all_product_strings:
    complete_dict[product] = np.linalg.multi_dot([clifford_generators[gen] for gen in product])
# now remove duplicates from the dictionary
unique_dict = {}
cosets = {}
for k, v in complete_dict.items():
    # if the transformation is not in the dictionary, add it
    if not any([np.allclose(v, u) for u in unique_dict.values()]):
        unique_dict[k] = v
        cosets[k] = [k]
    # else add it to the corresponding coset
    else:
        for u, w in unique_dict.items():
            if np.allclose(v, w):
                cosets[u].append(k)
                break
all_clifford_operations_1q = unique_dict

In [58]:
print(len(unique_dict), 'unique products')
print(len(complete_dict), 'total products')

24 unique products
256 total products


In [99]:
def clifford_gen_to_unitary(op):
    if op == 'ri':
        return np.eye(2)
    elif op == 'rx':
        return expm(-1j*(np.pi/4)*PauliMatrix(1))
    elif op == 'ry':
        return expm(-1j*(np.pi/4)*PauliMatrix(2))
    elif op == 'rz':
        return expm(-1j*(np.pi/4)*PauliMatrix(3))
    else:
        raise ValueError('Invalid Clifford operation')
    


def clifford_operations_to_unitary(op_list):
    if len(op_list) == 0:
        return np.eye(2)
    return np.linalg.multi_dot([clifford_gen_to_unitary(op) for op in op_list])

def clifford_ops_to_pmat_2q(ops1, ops2):
    return unitary_to_pmat(np.kron(clifford_operations_to_unitary(ops1), clifford_operations_to_unitary(ops2)))

    

In [100]:
# make the tensor product of all single qubit clifford rotations
all_separable_clifford_operations = {}
for k, v in all_clifford_operations_1q.items():
    for k2, v2 in all_clifford_operations_1q.items():
        all_separable_clifford_operations[(k, k2)] = clifford_ops_to_pmat_2q(k, k2)

In [101]:
list_all_clifford_operators = list(all_separable_clifford_operations.values())
list_all_clifford_strings = list(all_separable_clifford_operations.keys())

In [116]:
list_all_clifford_strings[2]

(('ri', 'ri', 'ri', 'ri'), ('ri', 'ri', 'ri', 'ry'))

In [118]:
PIY = unitary_to_pmat(np.kron(np.eye(2), expm(-1j*(np.pi/4)*PauliMatrix(2))))

In [119]:
pIY = expm((np.pi/2)*make_generator(np.kron(np.eye(2), PauliMatrix(2))))
np.all(np.isclose(pIY, PIY))

True

In [121]:
np.all(np.isclose(list_all_clifford_operators[2], PIY))

True

In [122]:
# make all the hamiltonian generators
all_hamiltonian_generators = {}
for i in range(16):
    pstring = decimal_to_pauli_string(i, 2)
    generator = make_generator(decimal_to_pauli_tensor(i, 2))
    all_hamiltonian_generators[pstring] = generator

In [110]:
pdict = make_pertubation_dict(zz_pmat, all_hamiltonian_generators, all_separable_clifford_operations)

In [111]:
def find_max_perturbation(pdict, generator_string):
    max_perturb = 0
    max_interleaving_op = -1
    for interleaving_op_string in pdict.keys():
        perturbation = pdict[interleaving_op_string][generator_string]
        perturb = max(abs(perturbation))
        if perturb > max_perturb:
            max_perturb = perturb
            max_interleaving_op = interleaving_op_string
    return max_perturb, max_interleaving_op

def rank_top_perturbations(pdict, generator_string, num_top_perturbations=5):
    perturbations = []
    for interleaving_op_string in pdict.keys():
        perturbation = pdict[interleaving_op_string][generator_string]
        perturbations.append((interleaving_op_string, max(abs(perturbation))))
    perturbations.sort(key=lambda x: x[1], reverse=True)
    return perturbations[:num_top_perturbations]

def list_all_perterbations(pdict, generator_string):
    perturbations = []
    for interleaving_op_string in pdict.keys():
        perturbation = pdict[interleaving_op_string][generator_string]
        perturbations.append((interleaving_op_string, max(abs(perturbation))))
    return perturbations

In [115]:
rank_top_perturbations(pdict, 'IZ')

[((('ri', 'rx', 'ry', 'rx'), ('ri', 'ri', 'ri', 'rz')), 1.0042436740011402),
 ((('ri', 'ri', 'rx', 'rx'), ('ri', 'ri', 'ry', 'ry')), 1.0000000000000004),
 ((('ri', 'ri', 'ri', 'ri'), ('ri', 'ri', 'ri', 'ri')), 1.0),
 ((('ri', 'ri', 'ri', 'ri'), ('ri', 'ri', 'ri', 'rz')), 1.0),
 ((('ri', 'ri', 'ri', 'ri'), ('ri', 'ri', 'rz', 'rz')), 1.0)]

In [124]:
rank_top_perturbations(pdict, 'IX')

[((('ri', 'ri', 'ry', 'ry'), ('rx', 'ry', 'ry', 'ry')), 0.8288592924627116),
 ((('ri', 'ri', 'ry', 'rx'), ('ri', 'rx', 'rx', 'ry')), 0.7088392820310624),
 ((('ri', 'ri', 'ry', 'ry'), ('ri', 'ri', 'ri', 'ri')), 0.6997202016541697),
 ((('ri', 'ri', 'ry', 'ry'), ('ri', 'ri', 'rx', 'ry')), 0.6852547876175787),
 ((('rx', 'rx', 'ry', 'rx'), ('rx', 'ry', 'ry', 'ry')), 0.6671749968386071)]

In [128]:
rank_top_perturbations(pdict, 'ZX')

[((('rx', 'ry', 'rx', 'rx'), ('ri', 'ry', 'rx', 'rx')), 0.7901325599869612),
 ((('ri', 'ri', 'ry', 'ry'), ('ri', 'ri', 'ri', 'ri')), 0.6997202016541695),
 ((('rx', 'rx', 'ry', 'rx'), ('ri', 'rx', 'rx', 'rz')), 0.6818910898936869),
 ((('ri', 'ri', 'rx', 'ry'), ('ri', 'ri', 'rx', 'rx')), 0.6661628171789445),
 ((('ri', 'ri', 'rz', 'ry'), ('rx', 'rx', 'rx', 'ry')), 0.6611403091810973)]

In [126]:
rank_top_perturbations(pdict, 'IY')

[((('ri', 'rx', 'rx', 'rz'), ('ri', 'ri', 'rx', 'rz')), 0.6640025990105775),
 ((('ri', 'ri', 'rz', 'rz'), ('ri', 'rx', 'rx', 'ry')), 0.6514778878198156),
 ((('ri', 'ri', 'rx', 'rx'), ('ri', 'ri', 'ri', 'ri')), 0.6401526561337315),
 ((('rx', 'rx', 'ry', 'rx'), ('ri', 'rx', 'rx', 'rz')), 0.6103053557561993),
 ((('ri', 'ry', 'rx', 'rx'), ('ri', 'rx', 'ry', 'ry')), 0.5877287667320381)]

In [127]:
rank_top_perturbations(pdict, 'YI')

[((('ri', 'ri', 'rz', 'ry'), ('ri', 'ri', 'rx', 'rz')), 0.6973293617517571),
 ((('ri', 'ri', 'rx', 'rz'), ('ri', 'rx', 'rx', 'ry')), 0.6623319022504598),
 ((('ri', 'ri', 'rx', 'rx'), ('ri', 'rx', 'ry', 'rx')), 0.6591569270336107),
 ((('ri', 'rx', 'rx', 'rx'), ('ri', 'ry', 'ry', 'ry')), 0.633267891487636),
 ((('ri', 'rx', 'ry', 'rx'), ('rx', 'rx', 'rx', 'ry')), 0.5979846707678924)]

In [133]:
make_sorted_eigsystem(zz_pmat@all_separable_clifford_operations[('rx', 'ry', 'rx', 'rx'), ('ri', 'ry', 'rx', 'rx')])

(array([-0.80901699-5.87785252e-01j, -0.80901699-5.87785252e-01j,
        -0.80901699-5.87785252e-01j,  0.30901699-9.51056516e-01j,
         0.30901699-9.51056516e-01j,  0.30901699-9.51056516e-01j,
         1.        +3.46944695e-17j,  1.        +1.11022302e-16j,
         1.        +2.11636264e-16j,  1.        +2.22153025e-16j,
         0.30901699+9.51056516e-01j,  0.30901699+9.51056516e-01j,
         0.30901699+9.51056516e-01j, -0.80901699+5.87785252e-01j,
        -0.80901699+5.87785252e-01j, -0.80901699+5.87785252e-01j]),
 array([[ 2.73861279e-01-2.73861279e-01j, -5.07658363e-02+3.37801155e-02j,
          4.97109517e-02+5.90952182e-02j,  2.73861397e-01-2.73861160e-01j,
         -6.41173153e-02-1.52536299e-02j,  1.13885272e-02+6.05263071e-03j,
          3.80423749e-01+0.00000000e+00j,  5.09056603e-01+0.00000000e+00j,
          3.52664003e-01-7.38224591e-02j,  3.79966971e-02+3.35463651e-01j,
          2.25403118e-01+2.55797309e-01j,  1.00286351e-01+1.07780445e-02j,
          4.63391020

In [ ]:
|